# Final Project - Deepfake

Try to do a simple deepfake. In this exercise, you have to swap faces in images that have two faces.

You can use the following link:

Face Swap using OpenCV ( C++ / Python ) :
https://learnopencv.com/face-swap-using-opencv-c-python/

In this exercise, you will use some of the previous lessons such as Seamless Cloning, but some concepts such as Triangulation will need to be studied.

# pip install opencv-python
# pip install dlib
# pip install imutils

In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install dlib

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install imutils

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25891 sha256=9c1282cf6e93cb5b91b115af797236ef3dc034cf383ba1ac6c47d9eae1021780
  Stored in directory: c:\users\matin\appdata\local\pip\cache\wheels\5b\76\96\ad0c321506837bef578cf3008df3916c23018435a355d9f6b1
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


In [11]:
import cv2
import numpy as np
import dlib

# Load the detector and predictor from dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Function to extract facial landmarks
def get_landmarks(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    if len(faces) == 0:
        return None
    landmarks = predictor(gray, faces[0])
    points = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(68)]
    return np.array(points, dtype=np.int32)

# Function to calculate Delaunay triangulation
def calculate_delaunay_triangulation(points, convex_hull):
    rect = cv2.boundingRect(convex_hull)
    subdiv = cv2.Subdiv2D(rect)
    subdiv.insert(points.tolist())
    triangles = subdiv.getTriangleList()
    return np.array(triangles, dtype=np.int32)

# Function to warp triangles
def warp_triangle(img1, img2, tri1, tri2):
    # Find bounding box for each triangle
    r1 = cv2.boundingRect(tri1)
    r2 = cv2.boundingRect(tri2)

    # Offset points by left top corner of the respective rectangles
    tri1_cropped = []
    tri2_cropped = []
    for i in range(3):
        tri1_cropped.append(((tri1[i][0] - r1[0]), (tri1[i][1] - r1[1])))
        tri2_cropped.append(((tri2[i][0] - r2[0]), (tri2[i][1] - r2[1])))

    # Apply warpAffine to small rectangular patches
    img1_cropped = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
    warp_mat = cv2.getAffineTransform(np.float32(tri1_cropped), np.float32(tri2_cropped))
    img2_cropped = cv2.warpAffine(img1_cropped, warp_mat, (r2[2], r2[3]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)

    # Get mask for the triangle
    mask = np.zeros((r2[3], r2[2], 3), dtype=np.float32)
    cv2.fillConvexPoly(mask, np.int32(tri2_cropped), (1.0, 1.0, 1.0), 16, 0)

    # Copy triangular region of the rectangular patch to the output image
    img2_cropped = img2_cropped * mask

    # Ensure data types are compatible before addition
    img2_patch = img2[r2[1]:r2[1] + r2[3], r2[0]:r2[0] + r2[2]].astype(np.float32)
    img2_patch = img2_patch * ((1.0, 1.0, 1.0) - mask)
    img2_patch += img2_cropped

    # Convert back to uint8 before assigning to img2
    img2[r2[1]:r2[1] + r2[3], r2[0]:r2[0] + r2[2]] = img2_patch.astype(np.uint8)

# Main function for face swap
def face_swap(img1, img2):
    # Get landmarks for both images
    points1 = get_landmarks(img1)
    points2 = get_landmarks(img2)

    if points1 is None or points2 is None:
        print("No faces detected!")
        return None

    # Find convex hulls
    hull1 = cv2.convexHull(points1)
    hull2 = cv2.convexHull(points2)

    # Check if convex hulls are valid
    if hull1.size == 0 or hull2.size == 0:
        print("Error: Convex hull calculation failed!")
        return None

    # Calculate Delaunay triangulation
    triangles = calculate_delaunay_triangulation(points1, hull1)

    # Warp triangles from img1 to img2
    img1_warped = np.copy(img2)
    for t in triangles:
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])

        # Find corresponding triangles in both images
        tri1 = [points1[np.argmin(np.linalg.norm(points1 - pt, axis=1))] for pt in [pt1, pt2, pt3]]
        tri2 = [points2[np.argmin(np.linalg.norm(points2 - pt, axis=1))] for pt in [pt1, pt2, pt3]]

        # Warp triangles
        warp_triangle(img1, img1_warped, np.array(tri1), np.array(tri2))

    # Create mask for seamless cloning
    try:
        hull8U = [(p[0], p[1]) for p in hull2.reshape(-1, 2)]  # Ensure points are reshaped correctly
    except IndexError:
        print("Error: Invalid convex hull points!")
        return None

    mask = np.zeros_like(img2[:, :, 0])
    cv2.fillConvexPoly(mask, np.int32(hull8U), 255)

    # Find center of the mask
    try:
        rect = cv2.boundingRect(np.float32(hull8U))  # Ensure hull8U is valid
    except Exception as e:
        print(f"Error calculating bounding rectangle: {e}")
        return None

    center = ((rect[0] + rect[2] // 2), (rect[1] + rect[3] // 2))

    # Perform seamless cloning
    output = cv2.seamlessClone(np.uint8(img1_warped), img2, mask, center, cv2.NORMAL_CLONE)

    return output

# Load images
img1 = cv2.imread("D:/exam/Face1.jpg")  # Source face
img2 = cv2.imread("D:/exam/Face2.jpg")  # Target face

# Check if images are loaded successfully
if img1 is None or img2 is None:
    print("Error: Could not load one or both images!")
    exit()

# Perform face swap
result = face_swap(img1, img2)

if result is not None:
    # Display the result
    cv2.imshow("Face Swapped", result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Save the result
    cv2.imwrite("face_swapped_result.jpg", result)